Archivo de Jupyter para cálculo simbólico de elementos de matriz

In [93]:
import numpy as np
from scipy.special import erf
from sympy import *
from sympy.vector import CoordSys3D, laplacian

In [2]:
A = CoordSys3D('A', transformation='spherical') # Sistema coordenado esférico, centrado en A
B = CoordSys3D('B', transformation='spherical') # Sistema coordenado esférico, centrado en B

a = Symbol('a', positive=True) # exponente orbital Gaussiano

Gaussiana $\ket{1\mathrm{s}}$ sin normalizar

$$ \phi_{1\mathrm{s}}^{(A)} (\mathbf{r} - \mathbf{R}_A) = e^{-\alpha |\mathbf{r} - \mathbf{R}_A|^2} = e^{-\alpha \, r^2_A} $$

donde $r = |\mathbf{r} - \mathbf{R}|$.

Se busca calcular a la integral de la forma
$$
\int e^{-\alpha \, r^2} \left(- \frac{1}{2} \nabla^2\right) e^{-\alpha \, r^2}\, \mathrm{d}r
$$

In [50]:
gauss = E**(-a*(A.r**2)) # función gaussiana 1s sin normalizar
integrand = simplify( gauss * (-1/2) * laplacian(gauss) ) # integrando
integral = simplify(integrate(integrand, (A.r, 0, oo)))

In [4]:
integral

0.625*sqrt(2)*sqrt(pi)*sqrt(a)

Es decir,
$$
\int_{0}^{\infty} e^{-\alpha \, r^2} \left(- \frac{1}{2} \nabla^2\right) e^{-\alpha \, r^2}\, \mathrm{d}r
= \frac{5}{8} \sqrt{2 \pi \alpha}
= \boxed{\frac{5}{4} \sqrt{\frac{\pi \alpha}{2}}}
$$

Así pues, para la función Gaussiana $\ket{1\mathrm{s}}$ normalizada se tendrá:
$$
\left( \frac{2(2\alpha)^{3/4}}{\pi^{1/4}} \right)^2 \int_{0}^{\infty} e^{-\alpha \, r^2} \left(- \frac{1}{2} \nabla^2\right) e^{-\alpha \, r^2} \, \mathrm{d}r
= \left( \frac{4(2\alpha)^{3/2}}{\pi^{1/2}} \right) \frac{5}{4} \sqrt{\frac{\pi \alpha}{2}}
= \boxed{10 a^2}
$$

In [5]:
def coeff(a):
    """
        Coeficiente de normalización para función radial Gaussiana 1s
    a : exponente orbital Gaussiano
    """
    return 2 * (2*a)**(3/4) / pi**(1/4)

In [6]:
def R1s_(a, C):
    """
        Función radial Gaussiana 1s sin normalizar
    a : exponente orbital Gaussiano
    C : CoordSys3D(C,spherical)
    """
    return E**(-a*(C.r**2))

In [7]:
def R1s(a, C):
    """
        Función radial Gaussiana 1s normalizada
    a : exponente orbital Gaussiano
    C : CoordSys3D(C,spherical)
    """
    return coeff(a) * R1s_(a,C)

In [8]:
R1s(a, A)

3.36358566101486*a**0.75*exp(-A.r**2*a)/pi**0.25

<div class="alert alert-block alert-info">
    El factor de $3.3635[\cdots]$ que aparece en la expresión de arriba es $2 \cdot 2^{3/4} = 2^{7/4}$ .
</div>

In [9]:
integrand = simplify( R1s(a, A) * (-1/2) * laplacian(R1s(a, A)) ) # integrando
integral = simplify(integrate(integrand, (A.r, 0, oo)))

In [10]:
integral

7.07106781186547*sqrt(2)*a**2.0

<div class="alert alert-block alert-info">
    El factor de $7.0710[\cdots]$ que aparece en la expresión de arriba es $5\sqrt{2}$ .
</div>

---
3-contracción Gaussiana $\ket{1s}$

In [11]:
def RSTO3G_(d, a, C):
    """
        3-contracción Gaussiana 1s
        Sin factor de normalización de Gaussianas
    d : vector de coeficientes de contracción
    a : vector de exponentes orbitales Gaussianos
    C : CoordSys3D(C,spherical)
    """
    d1, d2, d3 = d
    a1, a2, a3 = a
    return d1*R1s_(a1,C) + d2*R1s_(a2,C) + d3*R1s_(a3,C)

In [12]:
def RSTO3G(d, a, C):
    """
        3-contracción Gaussiana 1s
        Con factor de normalización de Gaussianas
    d : vector de coeficientes de contracción
    a : vector de exponentes orbitales Gaussianos
    C : CoordSys3D(C,spherical)
    """
    d1, d2, d3 = d
    a1, a2, a3 = a
    return d1*R1s(a1,C) + d2*R1s(a2,C) + d3*R1s(a3,C)

In [13]:
a1, a2, a3 = symbols('a1 a2 a3', positive=True) # exponentes orbitales Gaussianos
d1, d2, d3 = symbols('d1 d2 d3', real=True) # coeficientes de contracción

dvec = [d1, d2, d3]
avec = [a1, a2, a3]

In [14]:
RSTO3G_(dvec, avec, A)

d1*exp(-A.r**2*a1) + d2*exp(-A.r**2*a2) + d3*exp(-A.r**2*a3)

---
Base espacial

Dado que una $k$-contracción poseé el mismo factor de normalización para cada una de las Gaussianas, se realizará la computación simbólica sin este ya que sería lo mismo que multiplicar a la contracción total por dicho factor después de la computación. Observe que la convención para declarar a una función `f` sin dicho factor de normalización es `f_` .

In [15]:
phiA_ = RSTO3G_(dvec, avec, A)
phiB_ = RSTO3G_(dvec, avec, B)

En general un elemento de matriz $f_{pq}$ será dado por
$$
f_{pq} = \int \chi_p (\hat{f}_1 + \hat{f}_2) \chi_q \, \mathrm{d}\mathbf{x}
    = \int \psi_p (\hat{f}_1 + \hat{f}_2) \psi_q \, \mathrm{d}\mathbf{r} \int Y_{00}^2 \, \mathrm{d}\Omega \int \sigma_p(\omega) \sigma_q(\omega) \, \mathrm{d}\omega
$$

La parte angular es 1 y la parte espinorial se trata de una delta de Kronecker, por lo cual solo los elementos de matriz $f_{pq}$ tal que sus partes espinoriales coincidan serán diferentes de cero.

Desarrollando la parte radial donde $\mathrm{d}\mathbf{r} = r^2$ es
$$
\begin{align}
\int \psi_p (\hat{f}_1 + \hat{f}_2) \psi_q \, \mathrm{d}\mathbf{r}
    &= \int (\varphi_A \pm_p \varphi_B) (\hat{f}_1 + \hat{f}_2) (\varphi_A \pm_q \varphi_B) \, \mathrm{d}\mathbf{r} \\
    &= (\varphi_A | \hat{f}_1 | \varphi_A) \pm_p (\varphi_B | \hat{f}_1 | \varphi_A) \pm_q (\varphi_A | \hat{f}_1 | \varphi_B) \pm_p \pm_q (\varphi_B | \hat{f}_1 | \varphi_B)
    + (\varphi_A | \hat{f}_2 | \varphi_A) \pm_p (\varphi_B | \hat{f}_2 | \varphi_A) \pm_q (\varphi_A | \hat{f}_2 | \varphi_B) \pm_p \pm_q (\varphi_B | \hat{f}_2 | \varphi_B)
\end{align}
$$

Por simplicidad, se empleará la convención de notación $f_{i}^{(CD)} = (\varphi_C | \hat{f}_i | \varphi_D)$ donde $i \in \{1,2\}$ y $C, D \in \{A,B\}$. Esto para cada uno de los términos del elemento de matriz. En cuyo caso, el elemento de matriz $f_{pq}$ luciría como
$$
\begin{align}
    f_{pq} &=
    f_{1}^{(AA)} \pm_p f_{1}^{(BA)} \pm_q f_{1}^{(AB)} \pm_p \pm_q f_{1}^{(BB)}
    + f_{2}^{(AA)} \pm_p f_{2}^{(BA)} \pm_q f_{2}^{(AB)} \pm_p \pm_q f_{2}^{(BB)}
\end{align}
$$

Recuerde que cada función $\varphi_C$ se trata de una $k$-contracción Gaussiana $1\mathrm{s}$ centrada en $C$. Más aún, note que las funciones $\varphi_A$ y $\varphi_B$ comparten el mismo conjunto de parámetros Gaussianos $\{ \mathbf{d}, \mathbf{a} \}$ y su única diferencia es estar centradas en núcleos distintos. Sin embargo, matemáticamente el estar centrada en otro núcleo no es más que un etiquetado, i.e. $A$ y $B$ son mudas. Por lo tanto, $f_{1}^{(AA)} = f_{1}^{(BB)}$, $f_{1}^{(AB)} = f_{1}^{(BA)}$, $f_{2}^{(AA)} = f_{2}^{(BB)}$ y $f_{2}^{(AB)} = f_{2}^{(BA)}$, así pues el elemento de matriz $f_{pq}$ será:

$$ f_{pq} = 2\delta_{pq} \left[ f_{1}^{(AA)} \pm_p f_{1}^{(AB)} + f_{2}^{(AA)} \pm_p f_{2}^{(AB)} \right] $$

Entonces los elementos de matriz fuera de la diagonal $f_{pq}$, aunque $p$ y $q$ sean ambos pares o nones, son cero. Es decir, solo los elementos en la diagonal son diferentes de cero.

$$ f_{pp} = 2 \left[ f_{1}^{(AA)} \pm_p f_{1}^{(AB)} + f_{2}^{(AA)} \pm_p f_{2}^{(AB)} \right] $$

El término $f_{1}^{AA}$ puede ser integrado simbólicamente con facilidad y evaluado de la misma manera.

$$
f_{1}^{AA}
= (\varphi_A | \hat{f}_1 | \varphi_A) = \int r^2 \left( \sum_i^{k} d_i \phi_{i}^{(A)} \right) \hat{f}_1 \left( \sum_j^{k} d_j \phi_{j}^{(A)} \right) \mathrm{d}r
= \sum_i^{k} \sum_j^{k} d_i d_j \int r^2 \phi_{i}^{(A)} \hat{f}_1 \phi_{j}^{(A)} \mathrm{d}r
$$

Para los términos $\hat{f}_1 \phi_{j}^{(A)}$ se tiene que

$$
\hat{f}_1 \phi_{j}^{(A)} = \left( -\frac{1}{2} \nabla^2 \right) \exp(-\alpha_j r^2)
= -\frac{1}{2} \frac{1}{r^2} \frac{\partial}{\partial r} \left( r^2 \frac{\partial}{\partial r}\exp\left[-\alpha_j r^2 \right] \right)
= \alpha_j (3 - 2 \alpha_j r^2) \exp(-\alpha_j r^2)
= \alpha_j (3 - 2 \alpha_j r^2) \phi_{j}^{(A)}
$$

In [311]:
# corroboración trabajo previo
lap = simplify((-1/2) * laplacian(E**(-a2*(A.r**2))))
lap

a2*(-2.0*A.r**2*a2 + 3.0)*exp(-A.r**2*a2)

In [336]:
ind_f1AA = simplify((A.r**2) * E**(-a1*(A.r**2)) * (-1/2)*lap) # integrando
int_f1AA = simplify(integrate(ind_f1AA, (A.r,0,oo))) # integral
int_f1AA

-0.375*sqrt(pi)*a1*a2/(a1 + a2)**(5/2)

De este modo, el elemento de matriz $f_1^{AA}$ será dado por
$$
f_{1}^{AA} = \sum_i^{k} \sum_j^{k} d_i d_j \int r^2 \phi_{i}^{(A)} \alpha_j(3 - 2 \, \alpha_j \, r^2) \phi_{j}^{(A)} \mathrm{d}r
= -\frac{3}{8} \sqrt{\pi} \sum_i^{k} \sum_j^{k} d_i d_j \frac{\alpha_i \alpha_j}{(\alpha_i + \alpha_j)^{5/2}}
$$

In [341]:
# término f_1^AA del elemento de matriz f_pq, cómputo directo

def f1AA_term(di:float, dj:float, ai:float, aj:float):
    """
    di : i-ésimo coeficiente de contracción
    dj : j-ésimo coeficiente de contracción
    ai : i-ésimo exponente orbital Gaussiano
    aj : j-ésimo exponente orbital Gaussiano
    """
    return di*dj*ai*aj/np.power(ai+aj,5/2)
    
def f1AA(d, a):
    """
    d : vector de coeficientes de contracción [d1, d2, ..., dn]
    a : vector de exponentes orbitales Gaussianos [a1, a2, ..., an]
    """
    k = len(d)
    suma = 0
    for i in range(k):
        for j in range(k):
            suma += f1AA_term(d[i], d[j], a[i], a[j])
    return (-3/8)*np.sqrt(np.pi)*suma

In [346]:
print("Se fijan todos los parámetros igual a 1.")
print("Cómputo directo:  ", f1AA([1],[1]))
print("Cómputo simbólico:", int_f1AA.subs({a1:1, a2:1}).evalf())

Se fijan todos los parámetros igual a 1.
Cómputo directo:   -0.11749820037332814
Cómputo simbólico: -0.117498200373328


Observe que $f_{1}^{(AB)}$ constará de $k^2$ términos pues
$$
\begin{align}
f_{1}^{(AB)} &= (\varphi_A | \hat{f}_1 | \varphi_B)
= \int r^2 \left( \sum_i^{k} d_i \phi_{i}^{(A)} \right) \hat{f}_1 \left( \sum_j^{k} d_j \phi_{j}^{(B)} \right) \, \mathrm{d}r
= \sum_i^{k} \sum_j^{k} d_i d_j \int r^2 \phi_{i}^{(A)} \hat{f}_1 \phi_{j}^{(B)} \, \mathrm{d}r
\end{align}
$$

donde la función $\phi_m^{(C)}$ es una función Gaussiana $1\mathrm{s}$ dada por
$$
\phi_m^{(C)} = \exp(-\alpha_m |\mathbf{r} - \mathbf{R}_C|^2) = \exp(-\alpha_m r_C^2)
$$

Esta última integral tiene solución y es de la forma [2, pág. 412]:
$$
\begin{align}
    f_{1}^{(AB)}
    &= \sum_i^{k} \sum_j^{k} d_i d_j \frac{\alpha_i \alpha_j}{\alpha_i + \alpha_j} \left[ \frac{3 - 2\alpha_i \alpha_j}{\alpha_i + \alpha_j} |\mathbf{R}_A - \mathbf{R}_B|^2 \right] \left[ \frac{\pi}{\alpha_i + \alpha_j} \right]^{3/2} \exp \left[ -\frac{\alpha_i \alpha_j}{\alpha_i + \alpha_j}|\mathbf{R}_A - \mathbf{R}_B|^2 \right] \\
    &= \pi^{3/2} \sum_i^{k} \sum_j^{k} d_i d_j \frac{\alpha_i \alpha_j}{(\alpha_i + \alpha_j)^{7/2}} (3 - 2\alpha_i \alpha_j) |\mathbf{R}_A - \mathbf{R}_B|^2 \exp \left[ -\frac{\alpha_i \alpha_j}{\alpha_i + \alpha_j}|\mathbf{R}_A - \mathbf{R}_B|^2 \right]
\end{align}
$$

la expresión general para $f_{1}^{(AB)}$.

In [347]:
def f1AB_term(di:float, dj:float, ai:float, aj:float, RA, RB):
    """
    di : i-ésimo coeficiente de contracción
    dj : j-ésimo coeficiente de contracción
    ai : i-ésimo exponente orbital Gaussiano
    aj : j-ésimo exponente orbital Gaussiano
    RA : coord. núcleo A, [A1, A2, A3]
    RB : coord. núcleo B, [B1, B2, B3]
    """
    R = np.linalg.norm(np.array(RA)-np.array(RB), ord=1) # distancia interatómica al cuadrado
    return di*dj*ai*aj/np.power(ai+aj, 7/2)*(3-2*ai*aj)*R*np.exp(-ai*aj*R/(ai+aj))

In [348]:
def f1AB(d, a, RA, RB):
    """
    d : vector de coeficientes de contracción [d1, d2, ..., dn]
    a : vector de exponentes orbitales Gaussianos [a1, a2, ..., an]
    R : vector de núcleos [Ra, Rb] donde Rc = [C1, C2, C3]
    RA : coord. núcleo A, [A1, A2, A3]
    RB : coord. núcleo B, [B1, B2, B3]
    """
    suma = 0
    k = len(d)
    for i in range(k):
        for j in range(k):
            suma += f1AB_term(d[i], d[j], a[i], a[j], RA, RB)
    return np.power(np.pi, 3/2) * suma

Para $f_{2}^{(AA)}$ se tiene

$$
\begin{align}
f_{2}^{(AA)} = (\varphi_A | \hat{f}_2 | \varphi_A)
&= \int r^2 \left( \sum_i^{k} d_i \phi_{i}^{(A)} \right) \hat{f}_2 \left( \sum_j^{k} d_j \phi_{j}^{(A)} \right) \, \mathrm{d}r\\
&= \sum_i^{k} \sum_j^{k} d_i d_j \int r^2 \phi_{i}^{(A)}  \left( - \sum_C \frac{Z_C}{r_C} \right) \phi_{j}^{(A)} \, \mathrm{d}r\\
&= \sum_i^{k} \sum_j^{k} d_i d_j \int r^2 \phi_{i}^{(A)}  \left( - \sum_C \frac{Z_C}{r_C} \right) \phi_{j}^{(A)} \, \mathrm{d}r 
\end{align}
$$

In [305]:
(1/A.r) *simplify(E**(-a1*(A.r**2)) * E**(-a2*(A.r**2)))

exp(A.r**2*(-a1 - a2))/A.r

In [304]:
integrate((1/A.r) *simplify(E**(-a1*(A.r**2)) * E**(-a2*(A.r**2))), A.r)

Ei(A.r**2*a1*exp_polar(I*pi) + A.r**2*a2*exp_polar(I*pi))/2

In [306]:
integrate((1/A.r) *simplify(E**(-a1*(A.r**2)) * E**(-a2*(A.r**2))), (A.r, 0, oo))

oo

Para $f_{2}^{(AB)}$ se tiene
$$
\begin{align}
    f_{2}^{(AB)} &= (\varphi_A | \hat{f}_2 | \varphi_B)
    = \int \varphi_A \hat{f}_2 \varphi_B \, \mathrm{d}\mathbf{r} \\
    &= \int \left( \sum_i^{k} d_i \phi_{i}^{(A)} \right) \left( - \sum_C \frac{Z_C}{r_C} \right) \left( \sum_j^{k} d_j \phi_{j}^{(B)} \right) \, \mathrm{d}\mathbf{r}\\
    &= \sum_i^{k} \sum_j^{k} d_i d_j \int \phi_{i}^{(A)} \left( - \sum_C \frac{Z_C}{r_C} \right) \phi_{j}^{(B)} \, \mathrm{d}\mathbf{r} \\
    &= \sum_i^{k} \sum_j^{k} d_i d_j \sum_C \left\{ -\frac{2\pi}{\alpha_i + \alpha_j} Z_C \, \exp\left[ -\frac{\alpha_i \alpha_j}{\alpha_i + \alpha_j} |\mathbf{R}_A - \mathbf{R}_B|^2 \right]
    F_0 \left[ (\alpha_i + \alpha_j) |\mathbf{R}_P - \mathbf{R}_C|^2 \right] \right\} \\
    &= -2\pi \sum_i^{k} \sum_j^{k} \left\{ \frac{d_i d_j}{\alpha_i + \alpha_j} \exp\left[ -\frac{\alpha_i \alpha_j}{\alpha_i + \alpha_j} |\mathbf{R}_A - \mathbf{R}_B|^2 \right]
    \sum_C Z_C \,
    F_0 \left[ (\alpha_i + \alpha_j) |\mathbf{R}_P - \mathbf{R}_C|^2 \right] \right\}
\end{align}
$$

donde $C \in \{A, B\}$ es alguno de los dos núcleos posibles, $\mathbf{R}_P$ es un punto en la línea que une a los centros $A$ y $B$ dado por
$$
\begin{align}
    \mathrm{R}_P = \frac{\alpha_i \mathbf{R}_A + \alpha_j \, \mathbf{R}_B}{\alpha_i + \alpha_j}
\end{align}
$$

y $F_0$ es la función dada por
$$
\begin{align}
    F_0 (t) = \frac{1}{2} \sqrt{\frac{\pi}{t}} \mathrm{erf} (\sqrt{t})
\end{align}
$$

con $\mathrm{erf}$ la función de error.

In [247]:
def Rp(a, b, RA, RB):
    return (a*np.array(RA) + b*np.array(RB))/(a+b)

In [248]:
def F0(t):
    return (1/2)*np.sqrt(np.pi/t)*erf(np.sqrt(t))

In [ ]:
erf(np.sqrt((ai+aj)*Rpc))

In [265]:
def f2AB_term(di:float, dj:float, ai:float, aj:float, R, Z):
    """
    di : i-ésimo coeficiente de contracción
    dj : j-ésimo coeficiente de contracción
    ai : i-ésimo exponente orbital Gaussiano
    aj : j-ésimo exponente orbital Gaussiano
    Ra : coord. núcleo A, [A1, A2, A3]
    Rb : coord. núcleo B, [B1, B2, B3]
    Za : carga nuclear núcleo A
    Zb : carga nuclear núcleo B
    """
    # len(R) = len(Z) = 2
    Rab = np.linalg.norm(np.array(R[0])-np.array(R[1]), ord=1) # distancia interatómica al cuadrado, |RA-RB|^2
    RP = Rp(ai, aj, R[0], R[1])
    
    suma = 0
    for C in range(2):
        Rpc = np.linalg.norm(RP - R[C], ord=1) # distancia al cuadrado entre el centro P y C |RP-RC|^2
        suma += di*dj / (ai+aj) * np.exp(-ai*aj*Rab/(ai+aj)) * Z[C] * F0((ai+aj)*Rpc)
    
    return suma

In [266]:
def f2AB(d, a, RA, RB, ZA, ZB):
    """
    d : vector de coeficientes de contracción [d1, d2, ..., dn]
    a : vector de exponentes orbitales Gaussianos [a1, a2, ..., an]
    RA : coord. núcleo A, [A1, A2, A3]
    RB : coord. núcleo B, [B1, B2, B3]
    ZA : carga nuclear núcleo A
    ZB : carga nuclear núcleo B
    """
    # len(d) = len(a) = k
    suma = 0
    k = len(d)
    for i in range(k):
        for j in range(k):
            suma += f2AB_term(d[i], d[j], a[i], a[j], [RA, RB], [ZA, ZB])
            
    return -2*np.pi * suma

Finalmente, es posible calcular el elemento de matriz $f_{pp} = 2 \left[ f_{1}^{(AA)} \pm_p f_{1}^{(AB)} + f_{2}^{(AA)} \pm_p f_{2}^{(AB)} \right]$. Si $p=1,2$ entonces el signo es positivo, mientras que si $p=3,4$ el signo será negativo.

In [267]:
def fpp(p, d, a, RA, RB, ZA, ZB):
    """
        Elemento de matriz f_pp
    p : p-ésimo elemento de la base ordenada
    d : vector de coeficientes de contracción [d1, d2, ..., dn]
    a : vector de exponentes orbitales Gaussianos [a1, a2, ..., an]
    RA : coord. núcleo A, [A1, A2, A3]
    RB : coord. núcleo B, [B1, B2, B3]
    ZA : carga nuclear núcleo A
    ZB : carga nuclear núcleo B
    """
    term1AA = f1AA(d, a)
    term1AB = f1AB(d, a, RA, RB)
    term2AA = f2AB(d, a, RA, RA, ZA, ZB)
    term2AB = f2AB(d, a, RA, RB, ZA, ZB)

    if p in [1,2]:
        return 2*(term1AA + term1AB + term2AA + term2AB)
    else:
        return 2*(term1AA - term1AB + term2AA - term2AB)

In [292]:
d = [0.44471812476789035, 0.5352716544572346, 0.1543000507808527]
a = [0.10983311305458726, 0.40584959263313375, 2.227949029647934]

RA = [0, 0, 0]
RB = [1.401, 0, 0]
ZA = 1
ZB = 1

f11 = fpp(1, d, a, RA, RB, ZA, ZB)
f33 = fpp(3, d, a, RA, RB, ZA, ZB)

C:\Users\rodri\AppData\Local\Temp\ipykernel_8104\2241380236.py:2: RuntimeWarning: divide by zero encountered in scalar divide
  return (1/2)*np.sqrt(np.pi/t)*erf(np.sqrt(t))


In [296]:
2*(f1AA(d, a) + f1AB(d, a, RA, RB) - f2AB(d, a, RA, RB, ZA, ZB))

80.5208909805519

In [297]:
2*(f1AA(d, a) - f1AB(d, a, RA, RB) + f2AB(d, a, RA, RB, ZA, ZB))

-75.4771791595625

In [269]:
f2AB(d, a, RB, RB, ZA, ZB)

C:\Users\rodri\AppData\Local\Temp\ipykernel_8104\2241380236.py:2: RuntimeWarning: divide by zero encountered in scalar divide
  return (1/2)*np.sqrt(np.pi/t)*erf(np.sqrt(t))


nan